### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [13]:
# Get the unique count of players
player_cnt = len(purchase_data["SN"].unique())

# Create a df to display in required format
player_cnt_df = pd.DataFrame({"Total Players": [player_cnt]})
player_cnt_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [14]:
# get the unique item counts
item_cnt = len(purchase_data["Item Name"].unique())

# get the purchase count
purchase_cnt = len(purchase_data["Purchase ID"])

# get the avergae price
price_avg = purchase_data["Price"].mean()

# get the total revenue
total_rev = purchase_data["Price"].sum()

# Create a df to display in required format
summary_df = pd.DataFrame({"Number of Unique Items": [item_cnt], 
                            "Average Price": [price_avg], 
                            "Number of Purchases": [purchase_cnt],
                            "Total Revenue": [total_rev]})

# format the average price & total revenue
summary_df["Average Price"] = summary_df["Average Price"].map("${:.2f}".format)
summary_df["Total Revenue"] = summary_df["Total Revenue"].map("${:,}".format)
summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [15]:
# array(['Male', 'Other / Non-Disclosed', 'Female'], dtype=object)

# get the Male count & male count percentage
v_male = purchase_data.loc[purchase_data['Gender'] == "Male"]
v_male_cnt = len(v_male["SN"].unique())
v_male_percent = (v_male_cnt/player_cnt)*100

# get the Female count & female count percentage
v_female = purchase_data.loc[purchase_data['Gender'] == "Female"]
v_female_cnt = len(v_female["SN"].unique())
v_female_percent = (v_female_cnt/player_cnt)*100

# get the others count & others count percentage
v_other = purchase_data.loc[purchase_data['Gender'] == "Other / Non-Disclosed"]
v_other_cnt = len(v_other["SN"].unique())
v_other_percent = (v_other_cnt/player_cnt)*100

# create gender dictionary
gender_dict = {
    "Total Count": pd.Series([v_male_cnt, v_female_cnt, v_other_cnt], index=["Male", "Female", "Other / Non-Disclosed"]),
     "Percentage of Players": pd.Series([v_male_percent, v_female_percent, v_other_percent], index=["Male", "Female", "Other / Non-Disclosed"])
    }

# create df for gender dictionary
gender_df = pd.DataFrame(gender_dict)

# format Percentage of Players column
gender_df["Percentage of Players"] = gender_df["Percentage of Players"].map("{:.2f}%".format)

gender_df


,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [16]:
# perform group by on Gender to obtain purchase count, avg. purchase price, avg. purchase total per person
purchase_grp = purchase_data.groupby('Gender').agg(
    Purchase_Count=('Price', "count"),
    Average_Purchase_Price=('Price', "mean"),
    Total_Purchase_Value=('Price', sum),
    Purchase_per_Person=('SN', "nunique")
)

# Create a df
purchase_df = pd.DataFrame(purchase_grp)

# find out Avg Total Purchase per Person (agg function didnt allow me to do divide)
purchase_df["Purchase_per_Person"] = purchase_df["Total_Purchase_Value"]/purchase_df["Purchase_per_Person"]

# rename the columns to replace _ with space
purchase_df.rename(columns={"Purchase_Count":"Purchase Count", 
                                         "Average_Purchase_Price":"Average Purchase Price",
                                         "Total_Purchase_Value":"Total Purchase Value",
                                         "Purchase_per_Person":"Avg Total Purchase per Person"}, inplace=True)


# format the data of columns
purchase_df["Average Purchase Price"] = purchase_df["Average Purchase Price"].map("${:.2f}".format)
purchase_df["Total Purchase Value"] = purchase_df["Total Purchase Value"].map("${:,.2f}".format)
purchase_df["Avg Total Purchase per Person"] = purchase_df["Avg Total Purchase per Person"].map("${:.2f}".format)

purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [17]:
# Min age: 7
# Max age: 45

# create bins for age
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 45]

# Create the categories for the age bins
age_groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# create a new DF (do not override original DF)
demograph_data = purchase_data.copy()

# create new field "age groups"
demograph_data["Age Groups"] = pd.cut(demograph_data["Age"], age_bins, labels = age_groups, include_lowest=True)

# Age Groups < 10
v_10 = demograph_data.loc[demograph_data['Age Groups'] == "<10"]
v_10_cnt = len(v_10["SN"].unique())
v_10_percent = (v_10_cnt/player_cnt)*100

# Age Groups 10-14
v_10_14 = demograph_data.loc[demograph_data['Age Groups'] == "10-14"]
v_10_14_cnt = len(v_10_14["SN"].unique())
v_10_14_percent = (v_10_14_cnt/player_cnt)*100

# Age Groups 15-19
v_15_19 = demograph_data.loc[demograph_data['Age Groups'] == "15-19"]
v_15_19_cnt = len(v_15_19["SN"].unique())
v_15_19_percent = (v_15_19_cnt/player_cnt)*100

# Age Groups 20-24
v_20_24 = demograph_data.loc[demograph_data['Age Groups'] == "20-24"]
v_20_24_cnt = len(v_20_24["SN"].unique())
v_20_24_percent = (v_20_24_cnt/player_cnt)*100

# Age Groups 25-29
v_25_29 = demograph_data.loc[demograph_data['Age Groups'] == "25-29"]
v_25_29_cnt = len(v_25_29["SN"].unique())
v_25_29_percent = (v_25_29_cnt/player_cnt)*100

# Age Groups 30-34
v_30_34 = demograph_data.loc[demograph_data['Age Groups'] == "30-34"]
v_30_34_cnt = len(v_30_34["SN"].unique())
v_30_34_percent = (v_30_34_cnt/player_cnt)*100

# Age Groups 35-39
v_35_39 = demograph_data.loc[demograph_data['Age Groups'] == "35-39"]
v_35_39_cnt = len(v_35_39["SN"].unique())
v_35_39_percent = (v_35_39_cnt/player_cnt)*100

# Age Groups 40+
v_40 = demograph_data.loc[demograph_data['Age Groups'] == "40+"]
v_40_cnt = len(v_40["SN"].unique())
v_40_percent = (v_40_cnt/player_cnt)*100


# create demograph dictionary
demograph_dict = {
    "Total Count": pd.Series([v_10_cnt, v_10_14_cnt, v_15_19_cnt, v_20_24_cnt, v_25_29_cnt, v_30_34_cnt, v_35_39_cnt, v_40_cnt], 
                                index=[age_groups]),
    "Percentage of Players": pd.Series([v_10_percent, v_10_14_percent, v_15_19_percent, v_20_24_percent, v_25_29_percent, v_30_34_percent, v_35_39_percent, v_40_percent], 
                                         index=[age_groups])
    }


# create df for demograph dictionary
demograph_df = pd.DataFrame(demograph_dict)


# format Percentage of Players column
demograph_df["Percentage of Players"] = demograph_df["Percentage of Players"].map("{:.2f}%".format)

demograph_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [18]:
# Min age: 7
# Max age: 45

# create bins for age
pur_age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 45]

# Create the categories for the age bins
pur_age_groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# create a new DF (do not override original DF)
pur_age_data = purchase_data.copy()

# create new field "age groups"
pur_age_data["Age Ranges"] = pd.cut(pur_age_data["Age"], pur_age_bins, labels = pur_age_groups, include_lowest=True)

# perform group by on Gender to obtain purchase count, avg. purchase price, avg. purchase total per person
pur_age_grp = pur_age_data.groupby('Age Ranges').agg(
    Age_Purchase_Count=('Price', "count"),
    Age_Average_Purchase_Price=('Price', "mean"),
    Age_Total_Purchase_Value=('Price', sum),
    Age_Purchase_per_Person=('SN', "nunique")
)

# Create a df
pur_age_grp_df = pd.DataFrame(pur_age_grp)

# find out Avg Total Purchase per Person (agg function didnt allow me to do divide)
pur_age_grp_df["Age_Purchase_per_Person"] = pur_age_grp_df["Age_Total_Purchase_Value"]/pur_age_grp_df["Age_Purchase_per_Person"]

# rename the columns to replace _ with space
pur_age_grp_df.rename(columns={"Age_Purchase_Count":"Purchase Count", 
                                         "Age_Average_Purchase_Price":"Average Purchase Price",
                                         "Age_Total_Purchase_Value":"Total Purchase Value",
                                         "Age_Purchase_per_Person":"Avg Total Purchase per Person"}, inplace=True)

# format the data of columns
pur_age_grp_df["Average Purchase Price"] = pur_age_grp_df["Average Purchase Price"].map("${:.2f}".format)
pur_age_grp_df["Total Purchase Value"] = pur_age_grp_df["Total Purchase Value"].map("${:,.2f}".format)
pur_age_grp_df["Avg Total Purchase per Person"] = pur_age_grp_df["Avg Total Purchase per Person"].map("${:.2f}".format)

pur_age_grp_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [19]:
# perform group by on SN to obtain purchase count, avg. purchase price, total purchase value
spenders_grp = purchase_data.groupby('SN').agg(
    Purchase_Count=('Price', "count"),
    Average_Purchase_Price=('Price', "mean"),
    Total_Purchase_Value=('Price', sum)
)

# Create a df
spenders_df = pd.DataFrame(spenders_grp)

# sort the values of Total_Purchase_Value in decending order to get top spenders
spenders_df.sort_values("Total_Purchase_Value" , ascending=False, inplace=True)

# rename the columns to replace _ with space
spenders_df.rename(columns={"Purchase_Count":"Purchase Count", 
                            "Average_Purchase_Price":"Average Purchase Price",
                            "Total_Purchase_Value":"Total Purchase Value"}, inplace=True)


# format the data of columns
spenders_df["Average Purchase Price"] = spenders_df["Average Purchase Price"].map("${:.2f}".format)
spenders_df["Total Purchase Value"] = spenders_df["Total Purchase Value"].map("${:,.2f}".format)

spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [20]:
# perform group by on SN to obtain purchase count, item price, total purchase value
popular_grp = purchase_data.groupby(['Item ID', 'Item Name']).agg(
    Purchase_Count=('Price', "count"),
    Item_Price=('Price', "mean"),
    Total_Purchase_Value=('Price', sum)
)

# Create a df
popular_df = pd.DataFrame(popular_grp)

# sort the values of Total_Purchase_Value in decending order to get top spenders
popular_df.sort_values("Purchase_Count" , ascending=False, inplace=True)

# rename the columns to replace _ with space
popular_df.rename(columns={"Purchase_Count":"Purchase Count", 
                            "Item_Price":"Item Price",
                            "Total_Purchase_Value":"Total Purchase Value"}, inplace=True)

# format the data of columns
popular_df["Item Price"] = popular_df["Item Price"].map("${:.2f}".format)
popular_df["Total Purchase Value"] = popular_df["Total Purchase Value"].map("${:,.2f}".format)

popular_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [21]:
# Reset the df so that we can sort and get correct results
# if we try to sort without resseting df, it doesnt give expected results
popular_df = pd.DataFrame(popular_grp)

# sort the values of Total_Purchase_Value in decending order to get top spenders
popular_df.sort_values("Total_Purchase_Value" , ascending=False, inplace=True)

# rename the columns to replace _ with space
popular_df.rename(columns={"Purchase_Count":"Purchase Count", 
                            "Item_Price":"Item Price",
                            "Total_Purchase_Value":"Total Purchase Value"}, inplace=True)

# format the data of columns
popular_df["Item Price"] = popular_df["Item Price"].map("${:.2f}".format)
popular_df["Total Purchase Value"] = popular_df["Total Purchase Value"].map("${:,.2f}".format)

popular_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
